# Homework Starter — Stage 15: Orchestration & System Design
Complete the sections below. Keep your answers concise and focused on orchestration readiness.

## 1) Project Task Decomposition
List 4–8 tasks. Add more rows as needed.

In [1]:
from pathlib import Path
import pandas as pd
tasks = pd.DataFrame({
    'task': ['ingest', 'clean', 'train_or_score', 'report'],
    'inputs': ['/data/raw.ext', 'prices_raw.json', 'prices_clean.json', 'model.json'],
    'outputs': ['prices_raw.json', 'prices_clean.json', 'model.json', 'report.txt'],
    'idempotent': [True, True, True, True]
})
tasks

,task,inputs,outputs,idempotent
0,ingest,/data/raw.ext,prices_raw.json,True
1,clean,prices_raw.json,prices_clean.json,True
2,train_or_score,prices_clean.json,model.json,True
3,report,model.json,report.txt,True


## 2) Dependencies (DAG)
Describe dependencies and paste a small diagram if you have one.

In [2]:
dag = {
    'ingest': [],
    'clean': ['ingest'],
    'train_or_score': ['clean'],
    'report': ['train_or_score']
}
dag

{'ingest': [],
 'clean': ['ingest'],
 'train_or_score': ['clean'],
 'report': ['train_or_score']}

## 3) Logging & Checkpoints Plan
Specify what you will log and where you will checkpoint for each task.

In [3]:
logging_plan = pd.DataFrame({
    'task': ['ingest', 'clean', 'train_or_score', 'report'],
    'log_messages': [
        'start/end, rows, source URI',
        'start/end, rows in/out',
        'params, metrics',
        'artifact path'
    ],
    'checkpoint_artifact': [
        'prices_raw.json',
        'prices_clean.json',
        'model.json',
        'report.txt'
    ]
})
logging_plan

,task,log_messages,checkpoint_artifact
0,ingest,"start/end, rows, source URI",prices_raw.json
1,clean,"start/end, rows in/out",prices_clean.json
2,train_or_score,"params, metrics",model.json
3,report,artifact path,report.txt


## 4) Right-Sizing Automation
Which parts will you automate now? Which stay manual? Why?

*(Write your rationale here.)*

## 5) (Stretch) Refactor One Task into a Function + CLI
Use the templates below.

In [4]:
import argparse, json, logging, sys
from datetime import datetime
from pathlib import Path

def my_task(input_path: str, output_path: str) -> None:
    """Example task: read text/JSON -> transform -> write JSON.

    - If input is JSON, load it; otherwise read text.
    - Add run metadata (timestamp, input_size).
    - Write normalized JSON to output_path.
    """
    logging.info('[my_task] start')
    in_p = Path(input_path)
    if not in_p.exists():
        logging.warning('[my_task] input %s not found; creating a placeholder', input_path)
        in_p.parent.mkdir(parents=True, exist_ok=True)
        in_p.write_text('{"message":"placeholder"}', encoding='utf-8')

    raw = in_p.read_text(encoding='utf-8')
    try:
        payload = json.loads(raw)
    except json.JSONDecodeError:
        payload = {'raw_text': raw}

    result = {
        'payload': payload,
        'meta': {
            'timestamp': datetime.utcnow().isoformat(timespec='seconds') + 'Z',
            'input_path': str(in_p),
            'input_bytes': len(raw.encode('utf-8')),
        }
    }

    out_p = Path(output_path)
    out_p.parent.mkdir(parents=True, exist_ok=True)
    out_p.write_text(json.dumps(result, indent=2), encoding='utf-8')
    logging.info('[my_task] wrote %s (%d bytes)', out_p, len(out_p.read_bytes()))

def main(argv=None):
    parser = argparse.ArgumentParser(description='Homework task wrapper')
    parser.add_argument('--input', required=True)
    parser.add_argument('--output', required=True)
    args = parser.parse_args(argv)
    logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)])
    my_task(args.input, args.output)

if __name__ == '__main__':
    # Example simulated CLI in notebook:
    main(['--input', 'data/in.ext', '--output', 'data/out.json'])

INFO:root:[my_task] start
INFO:root:[my_task] wrote data\out.json (174 bytes)


C:\Users\Aislay\AppData\Local\Temp\ipykernel_18276\2349274777.py:28: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'timestamp': datetime.utcnow().isoformat(timespec='seconds') + 'Z',


### Optional: Simple Retry Wrapper (fill in)
Add a small retry with linear backoff to harden a task.

In [5]:
import time
def retry(n_tries=3, delay=0.2):
    def wrapper(fn, *args, **kwargs):
        # TODO: implement try/except loop with sleep backoff
        return fn(*args, **kwargs)
    return wrapper